# ETL pipelines with Dask

In [1]:
# Loading a big file
import dask.dataframe as dd

In [2]:
## Optionally create a client to visualize the processing
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 7.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39005,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:33541,Total threads: 1
Dashboard: http://127.0.0.1:40965/status,Memory: 1.87 GiB
Nanny: tcp://127.0.0.1:34137,


## Working with a dataframe of several GB!!

In [3]:
df = dd.read_csv("../data/2023_Yellow_Taxi_Trip_Data.csv", dtype={'improvement_surcharge': 'float64',
       'tolls_amount': 'float64'}, assume_missing=True)

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2.0,01/01/2023 12:32:10 AM,01/01/2023 12:40:36 AM,1.0,0.97,1.0,N,161.0,141.0,2.0,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2.0,01/01/2023 12:55:08 AM,01/01/2023 01:01:27 AM,1.0,1.10,1.0,N,43.0,237.0,1.0,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2.0,01/01/2023 12:25:04 AM,01/01/2023 12:37:49 AM,1.0,2.51,1.0,N,48.0,238.0,1.0,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1.0,01/01/2023 12:03:48 AM,01/01/2023 12:13:25 AM,0.0,1.90,1.0,N,138.0,7.0,1.0,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2.0,01/01/2023 12:10:29 AM,01/01/2023 12:21:19 AM,1.0,1.43,1.0,N,107.0,79.0,1.0,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
# Check the number of partitions
df.npartitions

59

In [6]:
# Convert the datetime column explicitely
df["tpep_pickup_datetime"] = df["tpep_pickup_datetime"].astype("datetime64[ns]")
df["tpep_dropoff_datetime"] = df["tpep_dropoff_datetime"].astype("datetime64[ns]")

In [7]:
# Creating a new column to calculate trip duration in minutes
df["trip_duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds()/60

In [8]:
# Filter out trips with zero duration
df = df[df.trip_duration >0]

In [ ]:
# Save the dataframe as PARQUET FORMAT
df.to_parquet("taxi_data", write_index=False)

/home/nicolas/.cache/pypoetry/virtualenvs/daskscratch-x6OfnjBx-py3.12/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/nicolas/.cache/pypoetry/virtualenvs/daskscratch-x6OfnjBx-py3.12/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/nicolas/.cache/pypoetry/virtualenvs/daskscratch-x6OfnjBx-py3.12/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
